# Few-Shot Model Evaluation

# 1. Initialize environment and model configuration

Import necessary libraries and modules from the torch and transformers packages. 

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

Define the path to the pre-trained model directory. Set up configurations for quantization, offering choices for both 4-bit and 8-bit quantization.

In [2]:
model_path = "../Model_pretrained/vicuna-7b-v1.5"

use_4bit = True
use_8bit = True
quant_type_4bit = "nf4"
quant_type_8bit = "nf8"
use_nested_quant = True
compute_dtype_bfloat16 = torch.bfloat16
compute_dtype_float16 = torch.float16
device_mapping = {"": 0}

""" Alternative parameters:
load_in_8bit_fp32_cpu_offload=True,  # Enable CPU offloading for certain layers
bnb_8bit_blocksparse_layout=None,  # Block-sparse layout, use None for dense models
bnb_8bit_custom_kernel=False,  # Use custom kernel, false by default
bnb_8bit_cpu_offload=True,  # Enable CPU offloading
bnb_8bit_cpu_offload_dtype=torch.float32,  # Data type for CPU offloaded tensors
bnb_8bit_cpu_offload_use_pin_memory=True,  # Use pinned memory for CPU offloading
bnb_8bit_cpu_offload_use_fast_fp32_to_fp16_conversion=False  # Use fast conversion from FP32 to FP16
"""

bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type = quant_type_4bit,
    bnb_4bit_use_double_quant = use_nested_quant,
    bnb_4bit_compute_dtype = compute_dtype_bfloat16
)
bnb_config_8bit = BitsAndBytesConfig(
    load_in_8bit = use_8bit,
    bnb_8bit_quant_type = quant_type_8bit,
    bnb_8bit_use_double_quant = use_nested_quant,
    bnb_8bit_compute_dtype = compute_dtype_float16
)

# 2. Load pretrained model

Load the tokenizer and model by selecting one of the configurations provided (4-bit or 8-bit).

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config = bnb_config_4bit, device_map = device_mapping)

bin C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\generation\configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Print the loaded model for inspection or verification.

In [4]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


# 3. Text generation pipeline: prompting test

Set up a text generation pipeline using the loaded model and tokenizer. Define a prompt for text generation related to a financial question. Generate text sequences based on the provided prompt using the text generation pipeline. Print the generated text sequences.

In [5]:
text_generation_pipeline = pipeline('text-generation', model = model, tokenizer = tokenizer)

prompt = """
I will provide you two examples related to question answering in the context of Financial Analysis of Current ratios:
Question_1: Current ratio of a company is 1.32 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 0.90, placing its value in the category defined as less than 1.00. What would be the Current ratio analysis for the years 2021 and 2022?
Answer_1: In 2021, the company had a Current ratio of 1.32, indicating that current assets were sufficient to cover short-term debts, though there were concerns about preparedness for unforeseen events affecting liquidity. However, in 2022, the ratio decreased to 0.90, falling below 1.00 and suggesting that current liabilities now exceed current assets, potentially posing challenges in meeting immediate financial obligations. The company should address these liquidity issues promptly, considering strategies to improving its financial position and enhance resilience in the face of unexpected financial needs.
Question_2: Current ratio of a company is 1.73 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has increased to 2.80, placing its value in the category defined as greater than 2.00. What would be the Current ratio analysis for the years 2021 and 2022?
Answer_2: The company's Current ratio increased from 1.73 in 2021 to 2.80 in 2022, reflecting a substantial improvement in its liquidity. In 2021, the Current ratio was in the 1.50 to 2.00 range, suggesting that the company had a balanced margin to cover its short-term obligations with its current asset and it was well-prepared to handle unforeseen events affecting liquidity. However, in 2022, the Current ratio was above 2.00, indicating an excess of liquidity. This implied that the company had more than enough current assets to cover its short-term liabilities. While this heightened liquidity provides a robust financial position, it is essential to strategically allocate these surplus resources by considering other alternatives such as making investments or reducing debt.
Now, give an answer to the following question: Current ratio of a company is 1.65 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has increased to 2.50, placing its value in the category defined as greater than 2.00. What would be the Current ratio analysis for the years 2021 and 2022?
Answer:
"""
sequences = text_generation_pipeline(
    prompt,
    max_length = 1024,
    do_sample = True,
    top_k = 10,
    temperature = 0.6,
    return_full_text = False,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: 
In 2021, the company's Current ratio was 1.65, indicating a balanced margin to cover short-term obligations with its current assets. This placed the company in a relatively stable position to handle unforeseen events affecting liquidity. However, in 2022, the Current ratio increased to 2.50, which is above the 2.00 threshold, suggesting an excess of liquidity. This implies that the company had more than enough current assets to cover its short-term liabilities. While this heightened liquidity provides a robust financial position, it is essential to strategically allocate these surplus resources by considering other alternatives such as making investments or reducing debt.


# 4. Setting DataFrames up

## 4.1 Load sample: formatting and splitting by ratio type

Import the pandas library and set an option to display the maximum column width to None (unlimited width). Define the path, file name and file type (extension) for the CSV file to be read. Read the CSV file into a pandas DataFrame using the specified path and file details. Display the first few rows of the DataFrame.

In [6]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

path = '../Sample_generation/Sample/'
file = 'sample'
type_file = '.csv'

sample = pd.read_csv(path+file+type_file, index_col = 0)

sample.head()

,instruction,analysis,ratio,company
0,"Current ratio of a company is 1.70 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.54, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.",Current,3M
1,"Quick ratio of a company is 1.08 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 0.93, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","It suggests that the company had a relatively healthy liquidity position in 2021. It implies that the company had sufficient liquid assets (such as cash, marketable securities, and accounts receivable, excluding inventory) to cover its short-term liabilities. In 2022 indicates that the liquidity position has weakened compared to the previous year. The quick ratio falling below 1.00 suggests that the company may have difficulty meeting its short-term obligations with its most liquid assets. It implies that the current assets, excluding inventory or other less liquid assets such Prepaid Expenses, may be insufficient to cover the current liabilities. The company might need to rely more on the sale of inventory or other sources to meet its immediate financial obligations.",Quick,3M
2,"Cash ratio of a company is 0.51 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.38, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The ratio for 2021 surpasses the upper limit, as cash constitutes more than 40% of current liabilities. This implies that the company may have an excess of cash on hand, surpassing what is required to meet its short-term obligations. In this context, there could be opportunities for the company to deploy the surplus cash more efficiently. However, it is worth noting that the ratio value for 2022 decreased and fell in the range between 0.20 and 0.40, coming into alignment with optimal cash levels.",Cash,3M
3,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.",Current,Adobe
4,"Quick ratio of a company is 1.11 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 1.00, placing its value in the category defined as between 1.00 and 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","The company had a healthy liquidity position in 2021. This implies that the company had sufficient liquid assets, such as cash, marketable securities, and accounts receivable (excluding inventory or other less liquid assets such prepaid expenses), to cover its short-term liabilities. In 2022 ratio suggests that the liquid

Define a function named format_and_split_df that takes the DataFrame as input and performs the following operations: Reindex the DataFrame with columns 'company', 'ratio', 'instruction', and 'analysis'. Rename the 'analysis' column to 'analysis_expected'. Add a new column 'analysis_generated' with all values set to None. Group the DataFrame by the 'ratio' column. Create three separate DataFrames based on the 'ratio' values (Current, Quick, Cash). Return a list containing these three DataFrames.

In [7]:
def format_and_split_df(df):
    dataframe = df.reindex(['company', 'ratio', 'instruction', 'analysis'], axis=1)
    dataframe = dataframe.rename(columns={'analysis': 'analysis_expected'})
    dataframe['analysis_generated'] = None
    
    grouped_df = dataframe.groupby('ratio')
    
    current_ratio_df = grouped_df.get_group('Current').reset_index(drop=True)
    quick_ratio_df = grouped_df.get_group('Quick').reset_index(drop=True)
    cash_ratio_df = grouped_df.get_group('Cash').reset_index(drop=True)
    
    return [current_ratio_df, quick_ratio_df, cash_ratio_df]

Unpack the list of DataFrames into three separate variables: current_ratio_df, quick_ratio_df, and cash_ratio_df. Display the first few rows of the resulting DataFrames.

In [8]:
dataframes = format_and_split_df(sample)

current_ratio_df, quick_ratio_df, cash_ratio_df = dataframes

In [9]:
current_ratio_df

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Current,"Current ratio of a company is 1.70 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.54, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.",None
1,Adobe,Current,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.",None
2,Airbnb,Current,"Current ratio of a company is 1.95 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.86, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the ratio remains within the ideal range, with short-term assets and receivables surpassing short-term obligations by 50% to 100%. This enables the company to effectively manage unforeseen events impacting liquidity. While the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.",None
3,Airbus,Current,"Current ratio of a company is 1.33 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.22, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the company possesses ample current assets to meet short-term debts, but there is a potential lack of readiness for unexpected events impacting liquidity. In 2022, the situation remains similar, but it is noteworthy that the ratio slightly decreases in value.",None
4,Amazon,Current,"Current ratio of a company is 1.14 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 0.94, placing its value in the category defined as less than 1.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, current assets (cash, accounts receivable, short-term investments and inventory) are enough to offset short-term debts. Nonetheless, the company might not be adequately prepared for unexpected events impacting its liquidity. In contrast, in 2022, ratio has decreased and is less than 1, signifying that current liabilities surpass current assets. This suggests a deficiency in liquidity, indicating that the company may struggle to meet its short-term debts.",None
5,Apple,Current,"Current ratio of a company is 1.07 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 0.88, placing its value in the category defined as less than 1.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021 the company would be 

## 4.2 Selecting rows for Few-shot prompting

Select samples that match all the existing ranges in the ratio for a better few-shot prompting. Display the rows and delete them from the DataFrame. Perform these operations for each ratio type.

### Current ratio

In [10]:
current_row_1 = current_ratio_df.loc[[34]]
current_row_1

,company,ratio,instruction,analysis_expected,analysis_generated
34,Otis,Current,"Current ratio of a company is 1.32 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 0.90, placing its value in the category defined as less than 1.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the company had a Current ratio of 1.32, indicating that current assets were sufficient to cover short-term debts, though there were concerns about preparedness for unforeseen events affecting liquidity. However, in 2022, the ratio decreased to 0.90, falling below 1.00 and suggesting that current liabilities now exceed current assets, potentially posing challenges in meeting immediate financial obligations. The company should address these liquidity issues promptly, considering strategies to improving its financial position and enhance resilience in the face of unexpected financial needs.",None


In [11]:
current_row_2 = current_ratio_df.loc[[42]]
current_row_2

,company,ratio,instruction,analysis_expected,analysis_generated
42,ResMed,Current,"Current ratio of a company is 1.73 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has increased to 2.80, placing its value in the category defined as greater than 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","The company's Current ratio increased from 1.73 in 2021 to 2.80 in 2022, reflecting a substantial improvement in its liquidity. In 2021, the Current ratio was in the 1.50 to 2.00 range, suggesting that the company had a balanced margin to cover its short-term obligations with its current asset and it was well-prepared to handle unforeseen events affecting liquidity. However, in 2022, the Current ratio was above 2.00, indicating an excess of liquidity. This implied that the company had more than enough current assets to cover its short-term liabilities. While this heightened liquidity provides a robust financial position, it is essential to strategically allocate these surplus resources by considering other alternatives such as making investments or reducing debt.",None


In [12]:
current_ratio_df = current_ratio_df.drop(34)
current_ratio_df = current_ratio_df.drop(42).reset_index(drop=True)
current_ratio_df

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Current,"Current ratio of a company is 1.70 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.54, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.",None
1,Adobe,Current,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.",None
2,Airbnb,Current,"Current ratio of a company is 1.95 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.86, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the ratio remains within the ideal range, with short-term assets and receivables surpassing short-term obligations by 50% to 100%. This enables the company to effectively manage unforeseen events impacting liquidity. While the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.",None
3,Airbus,Current,"Current ratio of a company is 1.33 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.22, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the company possesses ample current assets to meet short-term debts, but there is a potential lack of readiness for unexpected events impacting liquidity. In 2022, the situation remains similar, but it is noteworthy that the ratio slightly decreases in value.",None
4,Amazon,Current,"Current ratio of a company is 1.14 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 0.94, placing its value in the category defined as less than 1.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, current assets (cash, accounts receivable, short-term investments and inventory) are enough to offset short-term debts. Nonetheless, the company might not be adequately prepared for unexpected events impacting its liquidity. In contrast, in 2022, ratio has decreased and is less than 1, signifying that current liabilities surpass current assets. This suggests a deficiency in liquidity, indicating that the company may struggle to meet its short-term debts.",None
5,Apple,Current,"Current ratio of a company is 1.07 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 0.88, placing its value in the category defined as less than 1.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021 the company would be 

### Quick ratio

In [13]:
quick_row_1 = quick_ratio_df.loc[[34]]
quick_row_1

,company,ratio,instruction,analysis_expected,analysis_generated
34,Otis,Quick,"Quick ratio of a company is 1.22 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 0.81, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","The company's Quick ratio declined from 1.22 in 2021 to 0.81 in 2022. In 2021, the quick ratio fell within the range of 1.00 to 1.50, indicating that the company had a sufficient buffer of liquid assets (such as cash, marketable securities, and accounts receivable) to cover its short-term liabilities. In 2022, a Quick ratio below 1.00 raises concerns about the company's ability to cover current liabilities without relying on inventory or other less liquid assets such as prepaid expenses.",None


In [14]:
quick_row_2 = quick_ratio_df.loc[[41]]
quick_row_2

,company,ratio,instruction,analysis_expected,analysis_generated
41,RalphLauren,Quick,"Quick ratio of a company is 2.07 in 2021, placing its value in the category defined as greater than 1.50. In 2022, Quick ratio has decreased to 1.36, placing its value in the category defined as between 1.00 and 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","The company's Quick ratio decreased from 2.07 in 2021 to 1.36 in 2022, reflecting a notable decline in its liquidity. In 2021, the Quick ratio was greater than 1.50, suggesting that the company had a surplus of liquid assets beyond the adequate levels needed to cover short-term liabilities. This surplus revealed a potential for the company to allocate resources for other purposes, such as investments or debt reduction. However, in 2022, the Quick ratio fell into the range of 1.00 to 1.50, indicating that the company had a correct proportion of liquid assets to cover short-term obligations. The specified range suggests that its current assets, including cash, marketable securities and accounts receivable, are sufficient to cover its short-term liabilities.",None


In [15]:
quick_ratio_df = quick_ratio_df.drop(34)
quick_ratio_df = quick_ratio_df.drop(41).reset_index(drop=True)
quick_ratio_df

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Quick,"Quick ratio of a company is 1.08 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 0.93, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","It suggests that the company had a relatively healthy liquidity position in 2021. It implies that the company had sufficient liquid assets (such as cash, marketable securities, and accounts receivable, excluding inventory) to cover its short-term liabilities. In 2022 indicates that the liquidity position has weakened compared to the previous year. The quick ratio falling below 1.00 suggests that the company may have difficulty meeting its short-term obligations with its most liquid assets. It implies that the current assets, excluding inventory or other less liquid assets such Prepaid Expenses, may be insufficient to cover the current liabilities. The company might need to rely more on the sale of inventory or other sources to meet its immediate financial obligations.",None
1,Adobe,Quick,"Quick ratio of a company is 1.11 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 1.00, placing its value in the category defined as between 1.00 and 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","The company had a healthy liquidity position in 2021. This implies that the company had sufficient liquid assets, such as cash, marketable securities, and accounts receivable (excluding inventory or other less liquid assets such prepaid expenses), to cover its short-term liabilities. In 2022 ratio suggests that the liquidity position has weakened compared to the previous year. While the quick ratio is still between 1.00 and 1.50, the decrease raises concerns about the ability to cover short-term liabilities with the most liquid assets.",None
2,Airbnb,Quick,"Quick ratio of a company is 1.89 in 2021, placing its value in the category defined as greater than 1.50. In 2022, Quick ratio has decreased to 1.81, placing its value in the category defined as greater than 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","In 2021 and 2022, current assets (excluding inventories and other less liquid assets) are more than sufficient to meet short-term current liabilities. This implies that the company presents an excess of liquidity and there is an opportunity to allocate resources towards liquidity for alternative purposes. Anyway, the ratio has decreased in 2022 compared to 2021.",None
3,Airbus,Quick,"Quick ratio of a company is 0.36 in 2021, placing its value in the category defined as less than 1.00. In 2022, Quick ratio has decreased to 0.35, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","The company had substantial difficulty meeting its short-term obligations using its most liquid assets in 2021. In 2022, the company continues to face challenges in meeting short-term obligations with its most liquid assets and the situation is even worse because Quick ratio has slightly decreased.",None
4,Amazon,Quick,"Quick ratio of a company is 0.91 in 2021, placing its value in the category defined as less than 1.00. In 2022, Quick ratio has decreased to 0.72, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","The company had some difficulty meeting its short-term obligations using its most liquid assets in both years. In 2022 Quick ratio has decreased, which suggests a further decline in the liquidity position of the company. A Quick ratio below 1.00 implies that most liquid assets were insufficient to cover its short-term liabilities.",None
5,Apple,Quick,"Quick ratio of a company is 1.02 in 2021, placing its value in the category defi

### Cash ratio

In [16]:
cash_row_1 = cash_ratio_df.loc[[34]]
cash_row_1

,company,ratio,instruction,analysis_expected,analysis_generated
34,Otis,Cash,"Cash ratio of a company is 0.25 in 2021, placing its value in the category defined as between 0.20 and 0.40. In 2022, Cash ratio has decreased to 0.17, placing its value in the category defined as less than 0.20. What would be the Cash ratio analysis for the years 2021 and 2022?","In 2021, the company's Cash ratio of 0.25 fell within the range of 0.20 to 0.40, indicating a reasonably healthy liquidity position. This suggested the company had a sufficient amount of cash to cover a significant portion of its short-term liabilities and the organization is likely to meet its immediate financial obligations without significant challenges. However, in 2022, the cash ratio decreased to 0.17, dropping below the 0.20 threshold. A Cash ratio below 0.20 indicates a low level of liquidity and the company may face challenges in paying off its short-term debts.",None


In [17]:
cash_row_2 = cash_ratio_df.loc[[35]]
cash_row_2

,company,ratio,instruction,analysis_expected,analysis_generated
35,Paramount,Cash,"Cash ratio of a company is 0.66 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.26, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The company's Cash ratio decreased from 0.66 in 2021 to 0.26 in 2022. In 2021, Cash ratio value is above 0.40, which suggests an excess of liquidity and it is advisable for the company to explore alternative uses for the excess of cash, like strategic investments or debt reduction. In 2022, Cash ratio value is within the range of 0.20 to 0.40, indicating the company has a reasonable amount of cash on hand to cover a significant portion of its short-term liabilities and it is likely to meet its immediate financial obligations without significant challenges.",None


In [18]:
cash_ratio_df = cash_ratio_df.drop(34)
cash_ratio_df = cash_ratio_df.drop(35).reset_index(drop=True)
cash_ratio_df

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Cash,"Cash ratio of a company is 0.51 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.38, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The ratio for 2021 surpasses the upper limit, as cash constitutes more than 40% of current liabilities. This implies that the company may have an excess of cash on hand, surpassing what is required to meet its short-term obligations. In this context, there could be opportunities for the company to deploy the surplus cash more efficiently. However, it is worth noting that the ratio value for 2022 decreased and fell in the range between 0.20 and 0.40, coming into alignment with optimal cash levels.",None
1,Adobe,Cash,"Cash ratio of a company is 0.55 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.52, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Both in 2021 and 2022, cash exceeds appropriate levels, so short-term resources are not being used adequately. Although the ratio has slightly decreased in 2022, the excess of liquidity persists.",None
2,Airbnb,Cash,"Cash ratio of a company is 0.95 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.92, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the amount of cash surpasses 40% of short-term current liabilities. This suggests an opportunity to redirect resources for other purposes. It is important to note that, although the ratio has decreased in 2022, the excess liquidity situation has not been resolved.",None
3,Airbus,Cash,"Cash ratio of a company is 0.10 in 2021, placing its value in the category defined as less than 0.20. In 2022, Cash ratio has increased to 0.16, placing its value in the category defined as less than 0.20. What would be the Cash ratio analysis for the years 2021 and 2022?","The company's cash ratio has increased from 0.10 in 2021 to 0.16 in 2022, indicating a positive trend. However, both ratios fall below the recommended range of 0.20 to 0.40. In 2021, the low cash ratio of 0.10 suggests a potential difficulty in meeting immediate financial obligations, signaling a liquidity risk. The increase to 0.16 in 2022 is an improvement, moving closer to the acceptable range, but the company still lacks a reasonable amount of cash to comfortably cover its short-term liabilities.",None
4,Amazon,Cash,"Cash ratio of a company is 0.25 in 2021, placing its value in the category defined as between 0.20 and 0.40. In 2022, Cash ratio has increased to 0.35, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the cash levels constitute an optimal range, making up between 20% and 40% of the current liabilities. However, it is important to note that in 2022 Cash ratio has increased, which means that company liquidity positition is stronger than in 2021.",None
5,Apple,Cash,"Cash ratio of a company is 0.28 in 2021, placing its value in the category defined as between 0.20 and 0.40. In 2022, Cash ratio has decreased to 0.15, placing its value in the category defined as less than 0.20. What would be the Cash ratio analysis for the years 2021 and 2022?","In 2021, the company demonstrated an adequate cash to current liabilities ratio, as cash is between 20% and 40% of short-term liabilities. However, in 2022, Cash ratio decreased and fell below 0.20, indicating that cash on hand is insufficient to cover 20% of current liabilities.",None
6,Booking,Cash,"Cash ratio of a company is 1.78 in 2021, placing its value in th

# 5. Generate expected analysis

Import necessary modules from PyTorch for working with datasets and dataloaders. Define a custom dataset class named CustomDataset that inherits from Dataset. This class takes a DataFrame as input during initialization and implements methods for length (__len__) and getting individual items (__getitem__). It extracts the 'instruction' column from the DataFrame for each item.

Define a function prompt_template that takes two parameters: question_value and template_type. Depending on the value of template_type, it selects one of the three predefined templates related to financial ratios (Current ratio, Quick ratio, and Cash ratio) and returns it as a formatted string. Each template is created by selecting samples that match all the existing ranges in the ratio for a better few-shot prompting.

Define a function generate_analysis that takes a DataFrame (dataframe) and a text generation pipeline (text_generation_pipeline) as input. Inside the function: Create a CustomDataset using the DataFrame passed as input of the function. Use a DataLoader to load the dataset with a batch size of 1. Iterate over the DataLoader batches, extracting the 'question', return the prompt calling the prompt_template function and generate a corresponding ratio analysis using the LLM. Store the generated answer in the 'analysis_generated' column of the DataFrame.

In [30]:
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        question = self.dataframe.iloc[idx]['instruction']
        return {
            'question': question
        }

def prompt_template(question_value, template_type):
    templates = {
        'current_ratio': f"""
            I will provide you two examples related to question answering in the context of Financial Analysis of Current ratios:
            Question_1: Current ratio of a company is 1.32 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 0.90, placing its value in the category defined as less than 1.00. What would be the Current ratio analysis for the years 2021 and 2022?
            Answer_1: In 2021, the company had a Current ratio of 1.32, indicating that current assets were sufficient to cover short-term debts, though there were concerns about preparedness for unforeseen events affecting liquidity. However, in 2022, the ratio decreased to 0.90, falling below 1.00 and suggesting that current liabilities now exceed current assets, potentially posing challenges in meeting immediate financial obligations. The company should address these liquidity issues promptly, considering strategies to improving its financial position and enhance resilience in the face of unexpected financial needs.
            Question_2: Current ratio of a company is 1.73 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has increased to 2.80, placing its value in the category defined as greater than 2.00. What would be the Current ratio analysis for the years 2021 and 2022?
            Answer_2: The company's Current ratio increased from 1.73 in 2021 to 2.80 in 2022, reflecting a substantial improvement in its liquidity. In 2021, the Current ratio was in the 1.50 to 2.00 range, suggesting that the company had a balanced margin to cover its short-term obligations with its current asset and it was well-prepared to handle unforeseen events affecting liquidity. However, in 2022, the Current ratio was above 2.00, indicating an excess of liquidity. This implied that the company had more than enough current assets to cover its short-term liabilities. While this heightened liquidity provides a robust financial position, it is essential to strategically allocate these surplus resources by considering other alternatives such as making investments or reducing debt.
            Now, give an answer to the following question: {question_value}
            Answer:
        """,
        'quick_ratio': f"""
            I will provide you two examples related to question answering in the context of Financial Analysis of Quick ratios:
            Question_1: Quick ratio of a company is 1.22 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 0.81, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?
            Answer_1: The company's Quick ratio declined from 1.22 in 2021 to 0.81 in 2022. In 2021, the quick ratio fell within the range of 1.00 to 1.50, indicating that the company had a sufficient buffer of liquid assets (such as cash, marketable securities, and accounts receivable) to cover its short-term liabilities. In 2022, a Quick ratio below 1.00 raises concerns about the company's ability to cover current liabilities without relying on inventory or other less liquid assets such as prepaid expenses.
            Question_2: Quick ratio of a company is 2.07 in 2021, placing its value in the category defined as greater than 1.50. In 2022, Quick ratio has decreased to 1.36, placing its value in the category defined as between 1.00 and 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?
            Answer_2: The company's Quick ratio decreased from 2.07 in 2021 to 1.36 in 2022, reflecting a notable decline in its liquidity. In 2021, the Quick ratio was greater than 1.50, suggesting that the company had a surplus of liquid assets beyond the adequate levels needed to cover short-term liabilities. This surplus revealed a potential for the company to allocate resources for other purposes, such as investments or debt reduction. However, in 2022, the Quick ratio fell into the range of 1.00 to 1.50, indicating that the company had a correct proportion of liquid assets to cover short-term obligations. The specified range suggests that its current assets, including cash, marketable securities and accounts receivable, are sufficient to cover its short-term liabilities.
            Now, give an answer to the following question: {question_value}
            Answer:
        """,
        'cash_ratio': f"""
            I will provide you two examples related to question answering in the context of Financial Analysis of Cash ratios:
            Question_1: Cash ratio of a company is 0.25 in 2021, placing its value in the category defined as between 0.20 and 0.40. In 2022, Cash ratio has decreased to 0.17, placing its value in the category defined as less than 0.20. What would be the Cash ratio analysis for the years 2021 and 2022?
            Answer_1: In 2021, the company's Cash ratio of 0.25 fell within the range of 0.20 to 0.40, indicating a reasonably healthy liquidity position. This suggested the company had a sufficient amount of cash to cover a significant portion of its short-term liabilities and the organization is likely to meet its immediate financial obligations without significant challenges. However, in 2022, the cash ratio decreased to 0.17, dropping below the 0.20 threshold. A Cash ratio below 0.20 indicates a low level of liquidity and the company may face challenges in paying off its short-term debts.
            Question_2: Cash ratio of a company is 0.66 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.26, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?
            Answer_2: The company's Cash ratio decreased from 0.66 in 2021 to 0.26 in 2022. In 2021, Cash ratio value is above 0.40, which suggests an excess of liquidity and it is advisable for the company to explore alternative uses for the excess of cash, like strategic investments or debt reduction. In 2022, Cash ratio value is within the range of 0.20 to 0.40, indicating the company has a reasonable amount of cash on hand to cover a significant portion of its short-term liabilities and it is likely to meet its immediate financial obligations without significant challenges.
            Now, give an answer to the following question: {question_value}
            Answer:
        """
    }

    selected_template = templates.get(template_type, "Invalid template type")

    return selected_template

def generate_analysis(dataframe, text_generation_pipeline, select_template):
    dataset = CustomDataset(dataframe)
    
    dataloader = DataLoader(dataset, batch_size=1)

    for idx, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        question = batch['question'][0]
        
        prompt = prompt_template(question, select_template)
        
        sequences = text_generation_pipeline(
            prompt,
            max_length=1024,
            do_sample=True,
            top_k=10,
            temperature = 0.6,
            return_full_text=False,
        )
        
        generated_text = sequences[0]['generated_text']
        dataframe.at[idx, 'analysis_generated'] = generated_text

    return dataframe

Set up a text generation pipeline using the LLM model and tokenizer.

In [25]:
text_generation_pipeline = pipeline('text-generation', model = model, tokenizer = tokenizer)

## 5.1 Expected analyses: Current ratio

Call the generate_analysis function using the Current ratio DataFrame created in step 4 and the text generation pipeline for updating the DataFrame with the analysis generated by the LLM. Display resulting DataFrame and save it to a CSV file.

In [26]:
template = 'current_ratio'
current_ratio_df = generate_analysis(current_ratio_df, text_generation_pipeline, template)

  0%|          | 0/48 [00:00<?, ?it/s]

C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [27]:
current_ratio_df

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Current,"Current ratio of a company is 1.70 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.54, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.","\n In 2021, the company's Current ratio was 1.70, indicating a balanced margin to cover short-term obligations with its current assets. This placed the company in a relatively stable position. However, in 2022, the Current ratio decreased to 1.54, falling within the 1.50 to 2.00 range. This suggests that the company's liquidity has slightly deteriorated, with its current assets no longer being sufficient to cover all short-term liabilities. The company should closely monitor its liquidity position and consider strategies to improve its financial position, such as enhancing cash flow management, reducing liabilities, or exploring alternative financing options."
1,Adobe,Current,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.","\n In 2021, the company's Current ratio was 1.25, indicating that current assets were sufficient to cover short-term debts, though there were concerns about preparedness for unforeseen events affecting liquidity. However, in 2022, the ratio decreased to 1.11, falling within the range of 1.00 and 1.49. This suggests that the company's liquidity position has improved, as current liabilities are now being met with current assets. Nevertheless, it is essential to maintain a healthy Current ratio to ensure the company's ability to meet immediate financial obligations and adapt to unforeseen challenges. The company should continue to monitor its liquidity position and consider strategies to further enhance its financial resilience."
2,Airbnb,Current,"Current ratio of a company is 1.95 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.86, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the ratio remains within the ideal range, with short-term assets and receivables surpassing short-term obligations by 50% to 100%. This enables the company to effectively manage unforeseen events impacting liquidity. While the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.","\n In 2021, the company's Current ratio was 1.95, indicating a balanced margin to cover short-term obligations with its current assets. The ratio was within the 1.50 to 2.00 range, suggesting a well-prepared position to handle unforeseen events affecting liquidity. However, in 2022, the Current ratio decreased to 1.

In [28]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_current_ratio'
type_file = '.csv'

current_ratio_df.to_csv(path+file+type_file, index=True)

## 5.2 Expected analyses: Quick ratio

Call the generate_analysis function using the Quick ratio DataFrame created in step 4 and the text generation pipeline for updating the DataFrame with the analysis generated by the LLM. Display resulting DataFrame and save it to a CSV file.

In [31]:
template = 'quick_ratio'
quick_ratio_df = generate_analysis(quick_ratio_df, text_generation_pipeline, template)

  0%|          | 0/48 [00:00<?, ?it/s]

C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [32]:
quick_ratio_df

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Quick,"Quick ratio of a company is 1.08 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 0.93, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","It suggests that the company had a relatively healthy liquidity position in 2021. It implies that the company had sufficient liquid assets (such as cash, marketable securities, and accounts receivable, excluding inventory) to cover its short-term liabilities. In 2022 indicates that the liquidity position has weakened compared to the previous year. The quick ratio falling below 1.00 suggests that the company may have difficulty meeting its short-term obligations with its most liquid assets. It implies that the current assets, excluding inventory or other less liquid assets such Prepaid Expenses, may be insufficient to cover the current liabilities. The company might need to rely more on the sale of inventory or other sources to meet its immediate financial obligations.","\n The company's Quick ratio declined from 1.08 in 2021 to 0.93 in 2022. In 2021, the quick ratio fell within the range of 1.00 to 1.50, indicating that the company had a sufficient buffer of liquid assets (such as cash, marketable securities, and accounts receivable) to cover its short-term liabilities. In 2022, a Quick ratio below 1.00 raises concerns about the company's ability to cover current liabilities without relying on inventory or other less liquid assets such as prepaid expenses. This suggests that the company's liquidity has weakened, and it may face difficulties in meeting its short-term obligations."
1,Adobe,Quick,"Quick ratio of a company is 1.11 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 1.00, placing its value in the category defined as between 1.00 and 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","The company had a healthy liquidity position in 2021. This implies that the company had sufficient liquid assets, such as cash, marketable securities, and accounts receivable (excluding inventory or other less liquid assets such prepaid expenses), to cover its short-term liabilities. In 2022 ratio suggests that the liquidity position has weakened compared to the previous year. While the quick ratio is still between 1.00 and 1.50, the decrease raises concerns about the ability to cover short-term liabilities with the most liquid assets.","\n The company's Quick ratio decreased from 1.11 in 2021 to 1.00 in 2022. In 2021, the quick ratio fell within the range of 1.00 to 1.50, indicating that the company had a sufficient buffer of liquid assets (such as cash, marketable securities, and accounts receivable) to cover its short-term liabilities. In 2022, the Quick ratio remained within the same range, indicating that the company's liquid assets were still adequate to cover its short-term liabilities. The decrease in the Quick ratio from 2021 to 2022 suggests that the company's liquidity position may have weakened slightly, but it still maintained a sufficient buffer of liquid assets to meet its short-term obligations."
2,Airbnb,Quick,"Quick ratio of a company is 1.89 in 2021, placing its value in the category defined as greater than 1.50. In 2022, Quick ratio has decreased to 1.81, placing its value in the category defined as greater than 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","In 2021 and 2022, current assets (excluding inventories and other less liquid assets) are more than sufficient to meet short-term current liabilities. This implies that the company presents an excess of liquidity and there is an opportunity to allocate resources towards liquidity for alternative purposes. Anyway, the ratio has decreased in 2022 compared to 2021.","\n The company's Quick ratio d

In [33]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_quick_ratio'
type_file = '.csv'

quick_ratio_df.to_csv(path+file+type_file, index=True)

## 5.3 Expected analyses: Cash ratio

Call the generate_analysis function using the Cash ratio DataFrame created in step 4 and the text generation pipeline for updating the DataFrame with the analysis generated by the LLM. Display resulting DataFrame and save it to a CSV file.

In [34]:
template = 'cash_ratio'
cash_ratio_df = generate_analysis(cash_ratio_df, text_generation_pipeline, template)

  0%|          | 0/48 [00:00<?, ?it/s]

C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [35]:
cash_ratio_df

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Cash,"Cash ratio of a company is 0.51 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.38, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The ratio for 2021 surpasses the upper limit, as cash constitutes more than 40% of current liabilities. This implies that the company may have an excess of cash on hand, surpassing what is required to meet its short-term obligations. In this context, there could be opportunities for the company to deploy the surplus cash more efficiently. However, it is worth noting that the ratio value for 2022 decreased and fell in the range between 0.20 and 0.40, coming into alignment with optimal cash levels.","\n In 2021, the company's Cash ratio of 0.51 fell within the range of 0.40 to 0.60, indicating a reasonable level of liquidity. This suggested the company had a sufficient amount of cash to cover a significant portion of its short-term liabilities and the organization is likely to meet its immediate financial obligations without significant challenges. However, in 2022, the cash ratio decreased to 0.38, dropping below the 0.40 threshold. A Cash ratio below 0.40 indicates a low level of liquidity and the company may face challenges in paying off its short-term debts."
1,Adobe,Cash,"Cash ratio of a company is 0.55 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.52, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Both in 2021 and 2022, cash exceeds appropriate levels, so short-term resources are not being used adequately. Although the ratio has slightly decreased in 2022, the excess of liquidity persists.","\n In 2021, the company's Cash ratio of 0.55 fell within the range of 0.40 to 0.60, indicating a reasonable level of liquidity. This suggested the company had a sufficient amount of cash to cover a significant portion of its short-term liabilities and the organization is likely to meet its immediate financial obligations without significant challenges. However, in 2022, the cash ratio decreased to 0.52, still falling within the range of 0.40 to 0.60. The decrease in the cash ratio suggests that the company's liquidity position has weakened slightly, but it is still within a reasonable range. The company may want to consider strategies to improve its liquidity position, such as reducing its reliance on short-term debt or increasing its cash reserves."
2,Airbnb,Cash,"Cash ratio of a company is 0.95 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.92, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the amount of cash surpasses 40% of short-term current liabilities. This suggests an opportunity to redirect resources for other purposes. It is important to note that, although the ratio has decreased in 2022, the excess liquidity situation has not been resolved.","\n In 2021, the company's Cash ratio of 0.95 fell within the range of 0.40 to 0.95, indicating a high level of liquidity. This suggested the company had a significant amount of cash on hand, which is advisable for the company to explore alternative uses for the excess of cash, like strategic investments or debt reduction. In 2022, the cash ratio decreased to 0.92, still falling within the range of 0.40 to 0.95. This suggests that the company still has a high level of liquidity and it is advisable for the company to continue exploring alternative uses for the excess of cash."
3,Airbus,Cash,"Cash ratio of a company is 0.10 in 2021, placing its value in the category defined as less than 0.20. In 2022, Cash ratio has

In [36]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_cash_ratio'
type_file = '.csv'

cash_ratio_df.to_csv(path+file+type_file, index=True)

# 6. Sentence similarity with Sentence Transformers

Import necessary libraries such as SentenceTransformer, util from sentence_transformers, pandas and torch. Define a function named calculate_cosine_similarity that takes a DataFrame (df) and a Sentence Transformer model (model) as input: Iterate through rows of the DataFrame, calculating cosine similarity between the embeddings of "analysis_expected" and "analysis_generated" columns using the Sentence Transformer model. 
Append the cosine similarity values to a list and add a new column 'cosine_similarity' to the DataFrame. 
Return the modified DataFrame.

In [37]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

pd.set_option('display.max_colwidth', None)

def calculate_cosine_similarity(df, model):
    cosine_similarities = []
    
    for index, row in df.iterrows():
        expected_embedding = model.encode([row['analysis_expected']])
        generated_embedding = model.encode([row['analysis_generated']])
        cosine_similarity_matrix = util.cos_sim(torch.tensor(expected_embedding), torch.tensor(generated_embedding)).numpy()
        cosine_similarity = cosine_similarity_matrix[0][0]
        cosine_similarities.append(cosine_similarity)

    df['cosine_similarity'] = cosine_similarities

    return df

Load Sentence Transformer Model: Specify the Sentence Transformer model name (st_model_name) as 'sentence-transformers/all-mpnet-base-v2'. Create an instance of the Sentence Transformer model (st_model) using the specified model name.

In [38]:
st_model_name = 'sentence-transformers/all-mpnet-base-v2'
st_model = SentenceTransformer(st_model_name)

## 6.1 Sentence similarity: Current ratio

Load Current ratio DataFrame created in step 5 from CSV file. Perform cosine similarity calculation using the calculate_cosine_similarity function and the Sentence Transformer model. Save the resulting DataFrame with added 'cosine_similarity' column to new CSV file.

In [39]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_current_ratio'
type_file = '.csv'

fs_current_ratio = pd.read_csv(path+file+type_file, index_col = 0)

In [40]:
fs_current_ratio_similarity = calculate_cosine_similarity(fs_current_ratio, st_model)

In [41]:
fs_current_ratio_similarity

,company,ratio,instruction,analysis_expected,analysis_generated,cosine_similarity
0,3M,Current,"Current ratio of a company is 1.70 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.54, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.","\n In 2021, the company's Current ratio was 1.70, indicating a balanced margin to cover short-term obligations with its current assets. This placed the company in a relatively stable position. However, in 2022, the Current ratio decreased to 1.54, falling within the 1.50 to 2.00 range. This suggests that the company's liquidity has slightly deteriorated, with its current assets no longer being sufficient to cover all short-term liabilities. The company should closely monitor its liquidity position and consider strategies to improve its financial position, such as enhancing cash flow management, reducing liabilities, or exploring alternative financing options.",0.894476
1,Adobe,Current,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.","\n In 2021, the company's Current ratio was 1.25, indicating that current assets were sufficient to cover short-term debts, though there were concerns about preparedness for unforeseen events affecting liquidity. However, in 2022, the ratio decreased to 1.11, falling within the range of 1.00 and 1.49. This suggests that the company's liquidity position has improved, as current liabilities are now being met with current assets. Nevertheless, it is essential to maintain a healthy Current ratio to ensure the company's ability to meet immediate financial obligations and adapt to unforeseen challenges. The company should continue to monitor its liquidity position and consider strategies to further enhance its financial resilience.",0.906084
2,Airbnb,Current,"Current ratio of a company is 1.95 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.86, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the ratio remains within the ideal range, with short-term assets and receivables surpassing short-term obligations by 50% to 100%. This enables the company to effectively manage unforeseen events impacting liquidity. While the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.","\n In 2021, the company's Current ratio was 1.95, indicating a balanced margin to cover short-term obligations with its current assets. The ratio was within the 1.50 to 2.00 range, suggesting a well-prepared position to handle unforeseen events affecting liquidity. However, in 202

In [42]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_current_ratio_similarity'
type_file = '.csv'

fs_current_ratio_similarity.to_csv(path+file+type_file, index=True)

## 6.2 Sentence similarity: Quick ratio

Load Quick ratio DataFrame created in step 5 from CSV file. Perform cosine similarity calculation using the calculate_cosine_similarity function and the Sentence Transformer model. Save the resulting DataFrame with added 'cosine_similarity' column to new CSV file.

In [43]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_quick_ratio'
type_file = '.csv'

fs_quick_ratio = pd.read_csv(path+file+type_file, index_col = 0)

In [44]:
fs_quick_ratio_similarity = calculate_cosine_similarity(fs_quick_ratio, st_model)

In [45]:
fs_quick_ratio_similarity

,company,ratio,instruction,analysis_expected,analysis_generated,cosine_similarity
0,3M,Quick,"Quick ratio of a company is 1.08 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 0.93, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","It suggests that the company had a relatively healthy liquidity position in 2021. It implies that the company had sufficient liquid assets (such as cash, marketable securities, and accounts receivable, excluding inventory) to cover its short-term liabilities. In 2022 indicates that the liquidity position has weakened compared to the previous year. The quick ratio falling below 1.00 suggests that the company may have difficulty meeting its short-term obligations with its most liquid assets. It implies that the current assets, excluding inventory or other less liquid assets such Prepaid Expenses, may be insufficient to cover the current liabilities. The company might need to rely more on the sale of inventory or other sources to meet its immediate financial obligations.","\n The company's Quick ratio declined from 1.08 in 2021 to 0.93 in 2022. In 2021, the quick ratio fell within the range of 1.00 to 1.50, indicating that the company had a sufficient buffer of liquid assets (such as cash, marketable securities, and accounts receivable) to cover its short-term liabilities. In 2022, a Quick ratio below 1.00 raises concerns about the company's ability to cover current liabilities without relying on inventory or other less liquid assets such as prepaid expenses. This suggests that the company's liquidity has weakened, and it may face difficulties in meeting its short-term obligations.",0.911200
1,Adobe,Quick,"Quick ratio of a company is 1.11 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 1.00, placing its value in the category defined as between 1.00 and 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","The company had a healthy liquidity position in 2021. This implies that the company had sufficient liquid assets, such as cash, marketable securities, and accounts receivable (excluding inventory or other less liquid assets such prepaid expenses), to cover its short-term liabilities. In 2022 ratio suggests that the liquidity position has weakened compared to the previous year. While the quick ratio is still between 1.00 and 1.50, the decrease raises concerns about the ability to cover short-term liabilities with the most liquid assets.","\n The company's Quick ratio decreased from 1.11 in 2021 to 1.00 in 2022. In 2021, the quick ratio fell within the range of 1.00 to 1.50, indicating that the company had a sufficient buffer of liquid assets (such as cash, marketable securities, and accounts receivable) to cover its short-term liabilities. In 2022, the Quick ratio remained within the same range, indicating that the company's liquid assets were still adequate to cover its short-term liabilities. The decrease in the Quick ratio from 2021 to 2022 suggests that the company's liquidity position may have weakened slightly, but it still maintained a sufficient buffer of liquid assets to meet its short-term obligations.",0.929551
2,Airbnb,Quick,"Quick ratio of a company is 1.89 in 2021, placing its value in the category defined as greater than 1.50. In 2022, Quick ratio has decreased to 1.81, placing its value in the category defined as greater than 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","In 2021 and 2022, current assets (excluding inventories and other less liquid assets) are more than sufficient to meet short-term current liabilities. This implies that the company presents an excess of liquidity and there is an opportunity to allocate resources towards liquidity for alternative purposes. Anyway, the ratio has decreased in 2022 compared to 20

In [46]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_quick_ratio_similarity'
type_file = '.csv'

fs_quick_ratio_similarity.to_csv(path+file+type_file, index=True)

## 6.3 Sentence similarity: Cash ratio

Load Cash ratio DataFrame created in step 5 from CSV file. Perform cosine similarity calculation using the calculate_cosine_similarity function and the Sentence Transformer model. Save the resulting DataFrame with added 'cosine_similarity' column to new CSV file.

In [47]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_cash_ratio'
type_file = '.csv'

fs_cash_ratio = pd.read_csv(path+file+type_file, index_col = 0)

In [48]:
fs_cash_ratio_similarity = calculate_cosine_similarity(fs_cash_ratio, st_model)

In [49]:
fs_cash_ratio_similarity

,company,ratio,instruction,analysis_expected,analysis_generated,cosine_similarity
0,3M,Cash,"Cash ratio of a company is 0.51 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.38, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The ratio for 2021 surpasses the upper limit, as cash constitutes more than 40% of current liabilities. This implies that the company may have an excess of cash on hand, surpassing what is required to meet its short-term obligations. In this context, there could be opportunities for the company to deploy the surplus cash more efficiently. However, it is worth noting that the ratio value for 2022 decreased and fell in the range between 0.20 and 0.40, coming into alignment with optimal cash levels.","\n In 2021, the company's Cash ratio of 0.51 fell within the range of 0.40 to 0.60, indicating a reasonable level of liquidity. This suggested the company had a sufficient amount of cash to cover a significant portion of its short-term liabilities and the organization is likely to meet its immediate financial obligations without significant challenges. However, in 2022, the cash ratio decreased to 0.38, dropping below the 0.40 threshold. A Cash ratio below 0.40 indicates a low level of liquidity and the company may face challenges in paying off its short-term debts.",0.863762
1,Adobe,Cash,"Cash ratio of a company is 0.55 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.52, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Both in 2021 and 2022, cash exceeds appropriate levels, so short-term resources are not being used adequately. Although the ratio has slightly decreased in 2022, the excess of liquidity persists.","\n In 2021, the company's Cash ratio of 0.55 fell within the range of 0.40 to 0.60, indicating a reasonable level of liquidity. This suggested the company had a sufficient amount of cash to cover a significant portion of its short-term liabilities and the organization is likely to meet its immediate financial obligations without significant challenges. However, in 2022, the cash ratio decreased to 0.52, still falling within the range of 0.40 to 0.60. The decrease in the cash ratio suggests that the company's liquidity position has weakened slightly, but it is still within a reasonable range. The company may want to consider strategies to improve its liquidity position, such as reducing its reliance on short-term debt or increasing its cash reserves.",0.762087
2,Airbnb,Cash,"Cash ratio of a company is 0.95 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.92, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the amount of cash surpasses 40% of short-term current liabilities. This suggests an opportunity to redirect resources for other purposes. It is important to note that, although the ratio has decreased in 2022, the excess liquidity situation has not been resolved.","\n In 2021, the company's Cash ratio of 0.95 fell within the range of 0.40 to 0.95, indicating a high level of liquidity. This suggested the company had a significant amount of cash on hand, which is advisable for the company to explore alternative uses for the excess of cash, like strategic investments or debt reduction. In 2022, the cash ratio decreased to 0.92, still falling within the range of 0.40 to 0.95. This suggests that the company still has a high level of liquidity and it is advisable for the company to continue exploring alternative uses for the excess of cash.",0.794325
3,Airbus,Cash,"Cash ratio of a company is 0.10 in 2021, placing its value in the category defin

In [50]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_cash_ratio_similarity'
type_file = '.csv'

fs_cash_ratio_similarity.to_csv(path+file+type_file, index=True)

# 7. Evaluation results

## 7.1 Create a DataFrame template

Define the 'index' and 'columns' for a DataFrame with specific labels for rows and columns. Create an empty DataFrame using the specified index and columns. This dataframe will store the mean cosine similarity for each ratio type.

In [2]:
import pandas as pd

index = ['few_shot']
columns = ['current_ratio', 'quick_ratio', 'cash_ratio']

df_template = pd.DataFrame(index=index, columns=columns)

df_template

,current_ratio,quick_ratio,cash_ratio
few_shot,NaN,NaN,NaN


## 7.2 Calculate Cosine Similarity mean

Define a function named 'sentence_similarity_mean' that calculates the mean of the 'cosine_similarity' column in a given DataFrame and formats it with six decimal places. Define a function named 'store_evaluation' that updates the 'df_template' DataFrame with the mean cosine similarity for a specific ratio type using a given DataFrame.

In [3]:
def sentence_similarity_mean(df):
    mean = df['cosine_similarity'].sum() / len(df)
    formatted_mean = '{:.6f}'.format(mean)
    return formatted_mean
    
def store_evaluation(index, ratio_type, df_template, df_results):
    df_template.at[index, ratio_type] = sentence_similarity_mean(df_results)

## Current ratio

Retrieve the file containing cosine similarity results for Current ratio. Set the ratio name to be updated.

In [4]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_current_ratio_similarity'
type_file = '.csv'
ratio_name = 'current_ratio'

fs_current_results = pd.read_csv(path + file + type_file)

Call 'store_evaluation' to update the 'df_template' DataFrame with mean cosine similarity values for the 'current_ratio' column.

In [5]:
store_evaluation(index[0], ratio_name, df_template, fs_current_results)

## Quick ratio

Retrieve the file containing cosine similarity results for Quick ratio. Set the ratio name column to be updated.

In [6]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_quick_ratio_similarity'
type_file = '.csv'
ratio_name = 'quick_ratio'

fs_quick_results = pd.read_csv(path + file + type_file)

Call 'store_evaluation' to update the 'df_template' DataFrame with mean cosine similarity values for the 'quick_ratio' column.

In [7]:
store_evaluation(index[0], ratio_name, df_template, fs_quick_results)

## Cash ratio

Retrieve the file containing cosine similarity results for Cash ratio. Set the ratio name column to be updated.

In [8]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_cash_ratio_similarity'
type_file = '.csv'
ratio_name = 'cash_ratio'

fs_cash_results = pd.read_csv(path + file + type_file)

Call 'store_evaluation' to update the 'df_template' DataFrame with mean cosine similarity values for the 'cash_ratio' column.

In [9]:
store_evaluation(index[0], ratio_name, df_template, fs_cash_results)

## 7.3 Evaluation summary

Display the updated DataFrame template with cosine similarity means and save it to a CSV file.

In [10]:
df_template

,current_ratio,quick_ratio,cash_ratio
few_shot,0.897725,0.928372,0.909384


In [11]:
path = './Few_Shot_Evaluation/'
file = 'few_shot_similarity_summary'
type_file = '.csv'

df_template.to_csv(path+file+type_file, index=True)